# 📊 Análise Exploratória de Dados reais de filmes do IMDB  

Esse arquivo do Jupyter Notebook contempla o meu projeto realizado para apresentar uma análise exploratória de dados usando Linguagem Python, Linguagem SQL e Banco de Dados SQLite. Para tal usei dados reais disponíveis publicamente, dados sobre filmes no site [IMDB](https://www.imdb.com/?ref_=nv_home), que é um site de avaliação e classificação de filmes, séries e programas de TV.

Aqui nesse notebook, irei explicar o passo a passo utilizado para responder às perguntas indicadas no [arquivo README]() do diretório principal **README - substituir depois**, que foram:

- 1- Quais São as Categorias de Filmes Mais Comuns no IMDB?
- 2- Qual o Número de Títulos Por Gênero?
- 3- Qual a Mediana de Avaliação dos Filmes Por Gênero?
- 4- Qual a Mediana de Avaliação dos Filmes Em Relação ao Ano de Estréia?
- 5- Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?
- 6- Qual o Filme Com Maior Tempo de Duração? Calcule os Percentis.
- 7- Qual a Relação Entre Duração e Gênero?
- 8- Qual o Número de Filmes Produzidos Por País?
- 9- Quais São os Top 10 Melhores Filmes?
- 10- Quais São os Top 10 Piores Filmes?

## Instalando e Carregando Pacotes - preparando o ambiente

A **versão do Jupyter Python** que foi testada e aplicada o projeto a seguir foi **3.9.12**.

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


### Bibliotecas e pacotes utilizados

Para realizar a leitura e acesso ao banco de dados do IMDB, que foram baixados [nesse link](https://www.imdb.com/interfaces/) no dia 08/09/2022, irei utilizar a biblioteca Python **imdb-sqlite**, na versão 0.1.3, que está disponível no site da [PyPI](https://pypi.org/). Essa biblioteca importa arquivos IMDB TSV para um banco de dados SQLite e será fundamental para facilitar o tratamento e extruturação desses dados. A biblioteca citada, está disponível no link: [imdb-sqlite](https://pypi.org/project/imdb-sqlite/). 

In [4]:
# Instala o pacote imdb-sqlite no Jupyter Notebook:
!pip install -q imdb-sqlite

Além do imdb-sqlite, realizaremos a instalação de outra biblioteca que ainda não tinha utilizado, a **pycountry**, na versão 22.3.5, também disponivel no PyPI [neste link](https://pypi.org/project/pycountry/), que fornece os bancos de dados ISO para os padrões de nomes de: países, idiomas e moedas, principalmente e será interessante para nós acessarmos o nome dos países de forma mais rápida, sem precisar realizar nenhuma conversão manual. 

In [5]:
# Instala o pacote pycountry no Jupyter Notebook:
!pip install -q pycountry

Os demais pacotes e bibliotecas utilizados nesse projeto, já foram antes utilizados por mim ou já vieram instalados com o pacote do anaconda Python, um resumo sobre a utilização de cada uma delas está disponível no README principal desse diretório.

In [6]:
# Imports
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")